In [33]:
import numpy as np
import pandas as pd
def new_SA_simulate_hawkes_process(num_nodes=20, base_intensity=0.1, alpha=0.5, beta=1.0,  max_time=100,threshold=0.5):
    # Set random seed for reproducibility
    np.random.seed(42)

    # Define a function to simulate Poisson process for each edge
    def simulate_hawkes_ogata(base_intensity, alpha, beta, max_time):
        events = []
        current_time = 0
        last_event_time = -np.inf
        lambda_star = base_intensity

        while current_time < max_time:
            # Propose next event time using exponential inter-event time
            u = np.random.uniform()
            current_time += -np.log(u) / lambda_star
            
            # Calculate conditional intensity at proposed time
            conditional_intensity = base_intensity + alpha * np.sum(np.exp(-beta * (current_time - np.array(events))))

            # Accept or reject the proposed event time
            if np.random.uniform() <= conditional_intensity / lambda_star:
                events.append(current_time)
                last_event_time = current_time
                lambda_star = conditional_intensity + alpha
            else:
                lambda_star = conditional_intensity

        return events


    # Store events in a list
    events = []
    
    # Simulate the Hawkes process for each edge
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            if i != j:
                times = simulate_hawkes_ogata(base_intensity, alpha, beta, max_time)
                for t in times:
                    events.append((i, j, t))


    # Sort events by time
    events.sort(key=lambda x: x[2])

    # Initialize adjacency matrix
    adj_matrix = np.zeros((num_nodes, num_nodes))

    # Initialize cumulative probabilities
    cumulative_prob = np.zeros((num_nodes, num_nodes))

    # Store final events with event types in a list
    final_events = []

    # Track cumulative intensity
    cumulative_intensity = np.zeros((num_nodes, num_nodes))

    for event in events:
        u, v, time = event
        
        # Update cumulative intensity
        cumulative_intensity[u, v] += base_intensity * (time - (cumulative_intensity[u, v] / base_intensity))
        
        # Mask off the events where A_ij = 0 only if cumulative intensity does not exceed threshold
        if adj_matrix[u, v] == 1 or (adj_matrix[u, v] == 0 and cumulative_intensity[u, v] > threshold):
            final_events.append((u, v, time, 1))  # Type 1 event

        # Check if cumulative intensity exceeds the threshold
        if adj_matrix[u, v] == 0 and cumulative_intensity[u, v] > threshold:
            adj_matrix[u, v] = 1
            adj_matrix[v, u] = 1
            final_events.append((u, v, time, 0))  # Type 0 event
            # cumulative_intensity[u, v] = 0  # Reset cumulative intensity after threshold is exceeded
        
        # Simulate random update of the adjacency matrix just before the timestamp
        
        if adj_matrix[u, v] == 0 and np.random.rand() < 0.05:
            adj_matrix[u, v] = 1
            adj_matrix[v, u] = 1
            final_events.append((u, v, time, 0))  # Type 0 event


    # Convert to DataFrame for better readability
    final_events_df = pd.DataFrame(final_events, columns=['u', 'v', 'time', 'k'])
    final_events_df.to_csv('/simulated_data/newSA_hawkes_process_events_{}.csv'.format(threshold), index=False)

In [38]:
new_SA_simulate_hawkes_process(num_nodes=20, base_intensity=0.1, alpha=0.5, beta=1.0,  max_time=300,threshold=0.1)